<a href="https://colab.research.google.com/github/Jellyjellyjinjin/Daycon-car-object-detection-/blob/main/yolov5_Resume_freeze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content
!unzip /content/drive/MyDrive/open.zip

In [4]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"/content/input/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"/content/input/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [16]:
make_yolo_dataset(train_img, train_txt, "train")
make_yolo_dataset(valid_img, valid_txt, "valid")
make_yolo_dataset(sorted(glob.glob("/content/test/*")), None, "test")

100%|██████████| 3400/3400 [09:27<00:00,  5.99it/s]


In [17]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15930, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 15930 (delta 23), reused 26 (delta 9), pack-reused 15880
Receiving objects: 100% (15930/15930), 14.60 MiB | 21.88 MiB/s, done.
Resolving deltas: 100% (10922/10922), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.8/610.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [19]:
TRAIN = True

In [26]:
%%time
%cd /content/yolov5

if TRAIN:
    !python train.py --resume

/content/yolov5
train: weights=yolov5s.pt, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-211-g94e943e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.001, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, 

In [27]:
%cp /content/yolov5/f/content/drive/MyDrive/ATL/weight/freeze/yolov5x_result/weights/best.pt /content/drive/MyDrive/ATL/weight/resumeFreeze
%cp /content/yolov5/f/content/drive/MyDrive/ATL/weight/freeze/yolov5x_result/weights/last.pt /content/drive/MyDrive/ATL/weight/resumeFreeze

In [ ]:
!python detect.py --weights /content/drive/MyDrive/ATL/weight/resumeFreeze/best.pt --img 640  --source /content/input/test --save-conf --save-txt

In [34]:
submit = pd.read_csv('/content/sample_submission.csv', header=0)
submit.columns

Index(['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y',
       'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'],
      dtype='object')

In [38]:

import glob
import cv2
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [30]:
infer_txt_list = glob.glob('/content/yolov5/runs/detect/exp/labels/*.txt')

In [31]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_max, x_max, y_max, x_max, y_min, x_min, y_min

In [39]:
submit = pd.DataFrame(columns=['file_name', 'class_id', 'confidence', 'point1_x', 'point1_y', 'point2_x', 'point2_y', 'point3_x', 'point3_y', 'point4_x', 'point4_y'])

for txt in tqdm(infer_txt_list):
    with open(txt, 'r') as f:
        lines = f.readlines()
        base_file_name = txt.split('/')[-1].split('.')[0]
        img_height, img_width = cv2.imread('/content/yolov5/runs/detect/exp/' + base_file_name + '.png').shape[:2]
        for line in lines:
            file_name, class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = yolo_to_labelme(line, img_width, img_height, txt)
            submit = submit.append({'file_name':file_name, 'class_id':class_id, 'confidence':confidence, 'point1_x':point1_x, 'point1_y':point1_y, 'point2_x':point2_x, 'point2_y':point2_y, 'point3_x':point3_x, 'point3_y':point3_y, 'point4_x':point4_x, 'point4_y':point4_y}, ignore_index=True)

100%|██████████| 3395/3395 [03:17<00:00, 17.23it/s]


In [40]:
submit.to_csv('/content/drive/MyDrive/resumefreeze.csv', index=False)